# ትግርኛ - Real-time Tigrigna Autocomplete

Real-time word suggestions as you type Tigrigna text.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import re
import os
from collections import defaultdict

print("Loading Tigrigna autocomplete system...")

In [ ]:
# Load dictionary from file
def load_dictionary():
    words = set()
    
    # Try to load from the data directory
    try:
        with open('data/tigrigna_words.txt', 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    words.add(word)
        print(f"Loaded {len(words)} words from data/tigrigna_words.txt")
        return words
    except FileNotFoundError:
        pass
    
    # Try alternative path
    try:
        with open('tigrigna_dictionary.txt', 'r', encoding='utf-8') as f:
            for line in f:
                word = line.strip()
                if word:
                    words.add(word)
        print(f"Loaded {len(words)} words from tigrigna_dictionary.txt")
        return words
    except FileNotFoundError:
        pass
    
    print("Dictionary file not found. Please ensure the file exists.")
    return set()

dictionary = load_dictionary()
print(f"Dictionary ready with {len(dictionary)} words")

In [ ]:
# Build prefix index for fast lookups
def build_prefix_index(words):
    index = defaultdict(list)
    for word in words:
        for i in range(1, len(word) + 1):
            prefix = word[:i]
            index[prefix].append(word)
    
    # Sort each prefix list
    for prefix in index:
        index[prefix] = sorted(list(set(index[prefix])))
    
    return index

prefix_index = build_prefix_index(dictionary)
print(f"Built prefix index with {len(prefix_index)} entries")

In [ ]:
# Get suggestions for a prefix
def get_suggestions(prefix, max_count=10):
    if not prefix:
        return []
    
    # Get words that start with the prefix
    suggestions = []
    for word in dictionary:
        if word.startswith(prefix):
            suggestions.append(word)
    
    # Sort by length and alphabetically
    suggestions.sort(key=lambda x: (len(x), x))
    return suggestions[:max_count]

# Test the suggestion function
test_suggestions = get_suggestions("ሰላ")
print(f"Test suggestions for 'ሰላ': {test_suggestions[:5]}")

test_suggestions_a = get_suggestions("ኣ")
print(f"Test suggestions for 'ኣ': {test_suggestions_a[:5]}")

In [ ]:
# Create the user interface
def create_autocomplete_ui():
    # Title
    title = widgets.HTML(
        value="""
        <div style='text-align: center; margin: 20px 0;'>
            <h1 style='color: #1976d2; margin: 0;'>ትግርኛ</h1>
            <h2 style='color: #424242; margin: 5px 0;'>Real-time Tigrigna Autocomplete</h2>
            <p style='color: #666;'>Type Tigrigna text and get instant suggestions</p>
        </div>
        """
    )
    
    # Text input
    text_input = widgets.Textarea(
        placeholder='ኣብዚ ትግርኛ ጽሑፍካ ጽሓፍ...',
        layout=widgets.Layout(width='100%', height='100px')
    )
    
    # Suggestions output
    suggestions_output = widgets.Output()
    
    def show_suggestions(text):
        with suggestions_output:
            clear_output()
            
            if not text.strip():
                display(HTML("<div style='padding: 10px; color: #666;'>Start typing to see suggestions...</div>"))
                return
            
            # Get the last word being typed
            words = re.split(r'[\s፡።፣፤፥፧፦፨፠፟]+', text.strip())
            if not words:
                return
            
            current_word = words[-1]
            if not current_word:
                return
            
            suggestions = get_suggestions(current_word, 10)
            
            if suggestions:
                html = "<div style='border: 1px solid #ddd; border-radius: 5px; background: white;'>"
                html += f"<div style='background: #f5f5f5; padding: 8px; font-weight: bold;'>Suggestions for '{current_word}':</div>"
                
                for i, suggestion in enumerate(suggestions, 1):
                    if suggestion.startswith(current_word):
                        highlighted = f"<strong style='color: #1976d2;'>{current_word}</strong>{suggestion[len(current_word):]}"
                    else:
                        highlighted = suggestion
                    
                    html += f"<div style='padding: 6px 12px; border-bottom: 1px solid #eee;'>"
                    html += f"<span style='color: #666; margin-right: 8px;'>{i}.</span>{highlighted}"
                    html += "</div>"
                
                html += "</div>"
                display(HTML(html))
            else:
                display(HTML(f"<div style='padding: 10px; color: #999; font-style: italic;'>No suggestions found for '{current_word}'</div>"))
    
    def on_text_change(change):
        show_suggestions(change['new'])
    
    text_input.observe(on_text_change, names='value')
    
    return title, text_input, suggestions_output

# Create and display the interface
title, text_input, suggestions_output = create_autocomplete_ui()

display(title)
display(text_input)
display(suggestions_output)

# Initialize with welcome message
with suggestions_output:
    display(HTML("<div style='padding: 10px; color: #666;'>Start typing to see suggestions...</div>"))

print("Tigrigna autocomplete ready! Start typing to see suggestions.")